# SQL Join

# Video Link:
https://www.youtube.com/playlist?list=PLwnUv3sNB6_UN6vcOAWMklILhtQa7mGZh

## Installation

*   Install MySQL Server package
*   Start MySQL database server



In [1]:
%%capture
! sudo apt install mysql-server

In [2]:
! /etc/init.d/mysql start

 * Starting MySQL database server mysqld
   ...done.


## Create a test user

*   After the installation completed, you can change the root's password
*   Create user `test_user` for logging in from `localhost`, with password `test_p@ssw0rd`
*   Grant all permissions on any table in any database to user `test_user` for logging in from `localhost`
*   Apply changes on privileges



In [3]:
! mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY '8OQsfBWBP4'; FLUSH PRIVILEGES;"

ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)


In [4]:
! mysql -u root -p8OQsfBWBP4 -e "CREATE USER 'test_user'@'localhost' IDENTIFIED BY 'test_p@ssw0rd';"
! mysql -u root -p8OQsfBWBP4 -e "GRANT ALL PRIVILEGES ON *.* TO 'test_user'@'localhost';"
! mysql -u root -p8OQsfBWBP4 -e "FLUSH PRIVILEGES;"

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1396 (HY000) at line 1: Operation CREATE USER failed for 'test_user'@'localhost'
mysql: [Warning] Using a password on the command line interface can be insecure.
mysql: [Warning] Using a password on the command line interface can be insecure.


## Create database and tables

*   Use `mysql-connector-python`, Python driver for communicating with MySQL servers
*   Create and use `HR` database
*   Create table `Employees` and `Departments`
*   Insert 4 records to table `Employees` and 3 records to table `Departments`



In [5]:
%%capture
!pip install mysql-connector-python

In [6]:
import mysql.connector
import pandas as pd

# Database connection settings
HOST = "localhost"
USER = "test_user"
PASSWORD = "test_p@ssw0rd"
DATABASE = "HR"

# Connect to MySQL server
conn = mysql.connector.connect(host=HOST, user=USER, password=PASSWORD)
cursor = conn.cursor()

# Create database
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DATABASE}")
cursor.execute(f"USE {DATABASE}")

# Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS Employees (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    department_id INT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Departments (
    id INT PRIMARY KEY,
    department_name VARCHAR(50)
);
""")

# Clear old data incase we rerun colab
cursor.execute("DELETE FROM Employees")
cursor.execute("DELETE FROM Departments")

# Insert sample data
cursor.executemany("INSERT INTO Employees (id, name, department_id) VALUES (%s, %s, %s)", [
    (1, 'Alice', 1),
    (2, 'Bob', 2),
    (3, 'Charlie', 1),
    (4, 'David', None)
])

cursor.executemany("INSERT INTO Departments (id, department_name) VALUES (%s, %s)", [
    (1, 'HR'),
    (2, 'Finance'),
    (3, 'Engineering')
])

# Commit when finished
conn.commit()

Observe available data
*   Check data in `Employees` table.
*   Check data in `Departments` table.



In [7]:
! mysql -u test_user -ptest_p@ssw0rd HR -e 'SHOW TABLES'

mysql: [Warning] Using a password on the command line interface can be insecure.
+--------------+
| Tables_in_HR |
+--------------+
| Departments  |
| Employees    |
+--------------+


In [8]:
! mysql -u test_user -ptest_p@ssw0rd HR -e 'SELECT * FROM Employees'

mysql: [Warning] Using a password on the command line interface can be insecure.
+----+---------+---------------+
| id | name    | department_id |
+----+---------+---------------+
|  1 | Alice   |             1 |
|  2 | Bob     |             2 |
|  3 | Charlie |             1 |
|  4 | David   |          NULL |
+----+---------+---------------+


In [9]:
! mysql -u test_user -ptest_p@ssw0rd HR -e 'SELECT * FROM Departments'

mysql: [Warning] Using a password on the command line interface can be insecure.
+----+-----------------+
| id | department_name |
+----+-----------------+
|  1 | HR              |
|  2 | Finance         |
|  3 | Engineering     |
+----+-----------------+


## INNER JOIN

INNER JOIN joins two tables based on common column and returns rows that have **matching values in both tables**.

```
SELECT Employees.id, Employees.name, Departments.department_name
        FROM Employees
        INNER JOIN Departments ON Employees.department_id = Departments.id
```
selects rows from both tables if values of `department_id` of `Employees` table and `id` of `Departments` table are matched. Only rows that are common between `Employees` table and `Departments` table are included.




## LEFT JOIN

LEFT JOIN combines two tables based on common column and returns **all rows from the left table**, and the matching rows from the right table. The result is NULL from the right side, if there is no match.

```
SELECT Employees.id, Employees.name, Departments.department_name
        FROM Employees
        LEFT JOIN Departments ON Employees.department_id = Departments.id
```
selects rows where values of `department_id` of `Employees` table matches `id` of `Departments` table. All records from the left table (`Employees`) are returned, even if there are no match in the right table (`Departments`).

## RIGHT JOIN

RIGHT JOIN combines two tables based on common column and returns **all rows from the right table**, and the matching rows from the left table. The result is NULL from the left side, if there is no match.

```
SELECT Employees.id, Employees.name, Departments.department_name
        FROM Employees
        RIGHT JOIN Departments ON Employees.department_id = Departments.id
```
selects rows where values of `department_id` of `Employees` table matches `id` of `Departments` table. All records from the right table (`Departments`) are returned, even if there are no match in the left table (`Employees`).

In [10]:
import pandas as pd
# Create dictionary of join type as key, and SQL command as value
queries = {
    "INNER JOIN": """
        SELECT Employees.id, Employees.name, Departments.department_name
        FROM Employees
        INNER JOIN Departments ON Employees.department_id = Departments.id
    """,
    "LEFT JOIN": """
        SELECT Employees.id, Employees.name, Departments.department_name
        FROM Employees
        LEFT JOIN Departments ON Employees.department_id = Departments.id
    """,
    "RIGHT JOIN": """
        SELECT Employees.id, Employees.name, Departments.department_name
        FROM Employees
        RIGHT JOIN Departments ON Employees.department_id = Departments.id
    """
}

# Execute queries and display results
for join_type, query in queries.items():
    print(f"Executing {join_type} query:")
    cursor.execute(query)
    results = cursor.fetchall()
    df = pd.DataFrame(results)
    print(df)
    print("-" * 30)

# Close connection
cursor.close()
conn.close()


Executing INNER JOIN query:
   0        1        2
0  1    Alice       HR
1  2      Bob  Finance
2  3  Charlie       HR
------------------------------
Executing LEFT JOIN query:
   0        1        2
0  1    Alice       HR
1  2      Bob  Finance
2  3  Charlie       HR
3  4    David     None
------------------------------
Executing RIGHT JOIN query:
     0        1            2
0  3.0  Charlie           HR
1  1.0    Alice           HR
2  2.0      Bob      Finance
3  NaN     None  Engineering
------------------------------


## Equivalent merge in pandas

Pandas DataFrames can be merged with a database-style join.

**`dataframe.merge(right, how, left_on, right_on)`** updates the content of two DataFrame by merging them together, using the specified method(s).
*   `dataframe` : DataFrame to be merged into
*   `right` : DataFrame to merge with
*   `how` : Specifies how to merge. Default is `inner`.
*   `left_on` : Column or index level names to join on in the left DataFrame.
*   `right_on` : Column or index level names to join on in the right DataFrame.



**`dataframe.drop(labels=None, axis=0, index=None, columns=None)`** Remove rows or columns by specifying label names and corresponding axis, or by directly specifying index or column names.

*   `labels` : Index or column labels to drop.
*   `axis` : Whether to drop labels from the index (0 or 'index') or columns (1 or 'columns').
*   `index` : Alternative to specifying axis (`labels, axis=0` is equivalent to `index=labels`)
*   `columns` : Alternative to specifying axis (`labels, axis=1` is equivalent to `columns=labels`).




In [12]:
import mysql.connector
import pandas as pd

# Database connection settings
HOST = "localhost"
USER = "test_user"
PASSWORD = "test_p@ssw0rd"
DATABASE = "HR"

# Connect to MySQL and fetch data
conn = mysql.connector.connect(host=HOST, user=USER, password=PASSWORD, database=DATABASE)
cursor = conn.cursor()

# Load Employees table into a Pandas DataFrame
cursor.execute("SELECT * FROM Employees")
employees = pd.DataFrame(cursor.fetchall(), columns=['id', 'name', 'department_id'])

# Load Departments table into a Pandas DataFrame
cursor.execute("SELECT * FROM Departments")
departments = pd.DataFrame(cursor.fetchall(), columns=['id', 'department_name'])

# Close connection
cursor.close()
conn.close()

# Perform equivalent JOINs using Pandas
joins = {
    "INNER JOIN": pd.merge(employees, departments, how='inner', left_on='department_id', right_on='id'),
    "LEFT JOIN": pd.merge(employees, departments, how='left', left_on='department_id', right_on='id'),
    "RIGHT JOIN": pd.merge(employees, departments, how='right', left_on='department_id', right_on='id')
}

# Display results
for join_type, df in joins.items():
    print(f"\n--- {join_type} ---")
    print(df)


--- INNER JOIN ---
   id_x     name  department_id  id_y department_name
0     1    Alice            1.0     1              HR
1     2      Bob            2.0     2         Finance
2     3  Charlie            1.0     1              HR

--- LEFT JOIN ---
   id_x     name  department_id  id_y department_name
0     1    Alice            1.0   1.0              HR
1     2      Bob            2.0   2.0         Finance
2     3  Charlie            1.0   1.0              HR
3     4    David            NaN   NaN             NaN

--- RIGHT JOIN ---
   id_x     name  department_id  id_y department_name
0   1.0    Alice            1.0     1              HR
1   3.0  Charlie            1.0     1              HR
2   2.0      Bob            2.0     2         Finance
3   NaN      NaN            NaN     3     Engineering
